# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import re
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sqlalchemy import create_engine

import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mintk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mintk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mintk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mintk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
df = pd.read_sql_table('messages', engine)
X = df['message']
Y = df.iloc[:, 4:]

In [3]:
# Print the first 5 rows of X in full
for i in range(5):
    print(X[i])

Weather update - a cold front from Cuba that could pass over Haiti
Is the Hurricane over or is it not over
Looking for someone but no name
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
says: west side of Haiti, rest of the country today and tonight


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize text
    tokens = word_tokenize(text)

    # Lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return clean_tokens

In [5]:
tokenize(X[3])

['un',
 'report',
 'leogane',
 '80',
 '90',
 'destroyed',
 'hospital',
 'st',
 'croix',
 'functioning',
 'need',
 'supply',
 'desperately']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SGDClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Find unique values for columns
one_class = []
for col in Y.columns:
    # print(col, np.unique(Y[col]))
    if len(np.unique(Y[col])) < 2:
        print(col, np.unique(Y[col]))
        one_class.append(col)

# Drop columns with only one class
Y.drop(one_class, axis=1, inplace=True) #This is done so that the model does not break

child_alone [0]


In [8]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [9]:
# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000025CE5569440>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=SGDClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.
y_pred = pipeline.predict(X_test)
for i, col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.73      0.42      0.54      1564
           1       0.83      0.95      0.89      4935
           2       0.50      0.02      0.04        55

    accuracy                           0.82      6554
   macro avg       0.69      0.46      0.49      6554
weighted avg       0.80      0.82      0.80      6554

request
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      5474
           1       0.80      0.58      0.67      1080

    accuracy                           0.91      6554
   macro avg       0.86      0.77      0.81      6554
weighted avg       0.90      0.91      0.90      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000025CE5569440>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=SGDClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000025CE5569440>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=SGDClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf':

In [17]:
parameters =  {
    'vect__max_df': (0.5, 0.75, 1.0), # max_df=0.5 means "ignore terms that appear in more than 50% of the documents"
    # 'vect__ngram_range': ((1, 1), (1, 2)), # unigrams or bigrams
    'tfidf__use_idf': (True, False), # use idf or not
    'clf__estimator__alpha': (1e-2, 1e-3), # learning rate
    # 'clf__estimator__loss': ('hinge', 'log', 'modified_huber'),  # loss function
    'clf__estimator__penalty': ('l2', 'l1', 'elasticnet') # regularization
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2) # reduced cv to 2 to speed up training

In [18]:
# Train pipeline
cv.fit(X_train, y_train) # Takes 11 minutes to run with above parameters and cv=2

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000025CE5569440>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=SGDClassifier()))]),
             param_grid={'clf__estimator__alpha': (0.01, 0.001),
                         'clf__estimator__penalty': ('l2', 'l1', 'elasticnet'),
                         'tfidf__use_idf': (True, False),
                         'vect__max_df': (0.5, 0.75, 1.0)})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
# Test model
y_pred = cv.predict(X_test)
print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf__estimator__alpha': 0.001, 'clf__estimator__penalty': 'l2', 'tfidf__use_idf': False, 'vect__max_df': 1.0}


In [20]:
# Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.
for i, col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.78      0.09      0.16      1564
           1       0.77      0.99      0.87      4935
           2       0.00      0.00      0.00        55

    accuracy                           0.77      6554
   macro avg       0.52      0.36      0.34      6554
weighted avg       0.77      0.77      0.69      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5474
           1       0.86      0.43      0.57      1080

    accuracy                           0.89      6554
   macro avg       0.88      0.71      0.76      6554
weighted avg       0.89      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [52]:
# A custom transformer that extracts the length of the text data
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_series = pd.Series(X)  # Convert input to Series
        return pd.DataFrame(X_series.apply(lambda x: len(x)))  # Apply length function
    

In [53]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('text_length', TextLengthExtractor()) # Added new feature which is the length of the text
    ])),

    ('clf', MultiOutputClassifier(LogisticRegression())) # Changed to LogisticRegression because it is more accurate than SGDClassifier
])

In [54]:
# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x0000025CE5569440>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('text_length',
                                                 TextLengthExtractor())])),
                ('clf', MultiOutputClassifier(estimator=LogisticRegression()))])

In [55]:
# Test model
y_pred = pipeline.predict(X_test)
# accuracy = (y_pred == y_test).mean()
# print("\nAccuracy:", accuracy)

for i, col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.70      0.51      0.59      1564
           1       0.85      0.93      0.89      4935
           2       0.00      0.00      0.00        55

    accuracy                           0.82      6554
   macro avg       0.52      0.48      0.49      6554
weighted avg       0.81      0.82      0.81      6554

request
              precision    recall  f1-score   support

           0       0.92      0.98      0.94      5474
           1       0.82      0.54      0.65      1080

    accuracy                           0.90      6554
   macro avg       0.87      0.76      0.80      6554
weighted avg       0.90      0.90      0.90      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6532
           1       0.00      0.00      0.00        22

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

### 9. Export your model as a pickle file

In [25]:
# Export model as a pickle file
import pickle
pickle.dump(pipeline, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [57]:
pipeline.predict(['We need food and water!'])
# X_test[0]

array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)